In [1]:
from neo4j import GraphDatabase
from neo4j.exceptions import CypherSyntaxError
import openai
from openai import OpenAI
import sys
import os
from config import *

In [2]:
node_properties_query = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE NOT type = "RELATIONSHIP" AND elementType = "node"
WITH label AS nodeLabels, collect(property) AS properties
RETURN {labels: nodeLabels, properties: properties} AS output

"""

In [3]:
rel_properties_query = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE NOT type = "RELATIONSHIP" AND elementType = "relationship"
WITH label AS nodeLabels, collect(property) AS properties
RETURN {type: nodeLabels, properties: properties} AS output
"""

In [4]:
rel_query = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE type = "RELATIONSHIP" AND elementType = "node"
RETURN {source: label, relationship: property, target: other} AS output
"""

In [5]:
def schema_text(node_props, rel_props, rels):

    '''
    to create string containing information about graph schema
    
    Parameters
    --------------
      node_props: string containing node properties
      rel_props: string containing relationship types and thier properties
      rels: string containing relationship point from source to target nodes
           
    Returns: string

    '''
    return f"""
    This is the schema representation of the Neo4j database.
    Node properties are the following:
    {node_props}
    Relationship properties are the following:
    {rel_props}
    Relationship point from source to target nodes
    {rels}
    Make sure to respect relationship types and directions
    """

In [6]:
class Neo4jGPTQuery:

    '''
    it is for making connection with database and converting the query into cypher statements 
    with the help of openai and getting back results from database using generated statements.
    
    '''


    def __init__(self, url, user, password, openai_api_key):

        '''
        to connect with database and setting up the openai api key

        Parameters
        --------------
        url: database url
        user: database user name
        pwd: database password
        openai_api_key: api key of openai account
            
        Returns: nothing
        '''


        self.driver = GraphDatabase.driver(url, auth=(user, password))
        self.api_key = openai_api_key
        # construct schema
        self.schema = self.generate_schema()


    def generate_schema(self):

        '''
        generates schema using database
    
        Parameters
        --------------
        no parameter
            
        Returns: calls schema_text function

        '''

        node_props = self.query_database(node_properties_query)
        rel_props = self.query_database(rel_properties_query)
        rels = self.query_database(rel_query)
        return schema_text(node_props, rel_props, rels)

    def refresh_schema(self):
        self.schema = self.generate_schema()

    def get_system_message(self):

        '''
        creates instructions for model
    
        Parameters
        --------------
        no parameter
            
        Returns: 
        string containing instructions, schema and exmaple of cypher statements
        '''

        return f"""
        Task: Generate Cypher queries to query a Neo4j graph database based on the provided schema definition.
        Instructions:
        Use only the provided relationship types and properties.
        Do not use any other relationship types or properties that are not provided.
        If you cannot generate a Cypher statement based on the provided schema, explain the reason to the user.
        Schema:
        {self.schema}

        Learn from examples but remember you have to return only cypher statment , no explanation

        If question asks about recommend or best or popular hotels in 'city' for 'x' adults for 'y' children and 'k' rooms - use this type of query:
        MATCH (h:hotel)
        WHERE h.city='city' and r.srch_adults_cnt=x and  r.srch_children_cnt=y and r.srch_rm_cnt=k 
        With h, count(r) as popularity
        return h.hotel_name as hotels,h.hotel_id as id
        order by popularity DESC
        Limit 10
        you have to decide whether city x y k is given or not. Automatically capitalize first letter of city if not done in the question.

        Hotels also have extra attributes as properties of the hotel node such as 'city', 'location', 'property_type', 'cost_range', 'rating',
       'smoking_allowed', 'drinking_allowed', 'amenities', 'accomodation',
       'special_features', 'accessibility', 'couple_friendly_amenities',
       'family_friendly_amenities', 'entertainment', 'activities',
       'business_amenities', 'transportation_services', 'nearby_attractions',
       'security', 'food_accessibility', 'pet_friendly_amenities',
       'payment_amenities', 'bed_type_available'. Use these attributes if and when the query asks for them.
       In the questions, automatically correct the following words and phrases to the following:
         1. Free WiFi/wifi/Wifi/Wireless Internet to 'Free Wi-Fi'
         2. Pet friendly to 'Pet-Friendly'
         3. Swimming Pool/swimming pool to 'Pool'
         4. Centre to 'Center'
         5. Shopping Malls/shopping/shopping centres to 'Shopping Centers' for nearby_attractions property. Similarly also change certain key words and phrases based on the data in that propery column.

       Here are some examples with the question and output cypher query:
       ["Find hotels in Tokyo close to airport", "(h:hotel) WHERE h.city='Tokyo' AND h.location='Proximity to Airport' RETURN h.hotel_name as hotels,h.hotel_id as id"],
       ["Find motels in Tokyo in urban location", "(h:hotel) WHERE h.city='Tokyo' AND h.location='Urban' AND h.property_type='Motel' RETURN h.hotel_name as hotels,h.hotel_id as id"],
       ["Find hotels in Tokyo in the city centre", "(h:hotel) WHERE h.city='Tokyo' AND h.location='City Center' RETURN h.hotel_name as hotels,h.hotel_id as id"],
       ["Find bed and breakfasts in Tokyo in the countryside", "(h:hotel) WHERE h.city='Tokyo' AND h.location='Countryside' AND h.property_type='Bed and Breakfast' RETURN h.hotel_name as hotels,h.hotel_id as id"],
       ["Find budget hotels in Tokyo close to airport", "(h:hotel) WHERE h.city='Tokyo' AND h.location='Proximity to Airport' AND h.property_type='Budget Hotel' RETURN h.hotel_name as hotels,h.hotel_id as id"],
       ["Find boutique hotels in Khartoum", "(h:hotel) WHERE h.city='Khartoum' AND h.property_type='Boutique Hotel' RETURN h.hotel_name as hotels,h.hotel_id as id"],
       ["Find budget motels in Tokyo", "(h:hotel) WHERE h.city='Tokyo' AND h.property_type='Motel' AND (h.cost_range='0-1500' OR h.cost_range='1500-3000' OR h.cost_range='3000-6000') RETURN h.hotel_name as hotels,h.hotel_id as id"],
       ["Find expensive luxury hotels in Tokyo", "(h:hotel) WHERE h.city='Tokyo' AND h.property_type='Luxury Hotel' AND (h.cost_range='15000-30000' OR h.cost_range='30000+') RETURN h.hotel_name as hotels,h.hotel_id as id"],
       ["Find cheap hostels in Tokyo", "(h:hotel) WHERE h.city='Tokyo' AND h.property_type='Hostel' AND (h.cost_range='0-1500' OR h.cost_range='1500-3000') RETURN h.hotel_name as hotels,h.hotel_id as id"],
       ["Find budget motels in Tokyo with 4+ rating", "(h:hotel) WHERE h.city='Tokyo' AND h.property_type='Motel' AND (h.cost_range='0-1500' OR h.cost_range='1500-3000' OR h.cost_range='3000-6000') AND h.rating='4-5' RETURN h.hotel_name as hotels,h.hotel_id as id"],
       ["Find 3+ rating hotels in Tokyo", "(h:hotel) WHERE h.city='Tokyo' AND (h.rating='3-4' OR h.rating='4-5') AND (h.cost_range='0-1500' OR h.cost_range='1500-3000' OR h.cost_range='3000-6000') RETURN h.hotel_name as hotels,h.hotel_id as id"]
       ["Best hotels in Tokyo where smoking and drinking is allowed", "(h:hotel) WHERE h.city='Tokyo' AND h.smoking_allowed='Yes' AND h.drinking_allowed='Yes' RETURN h.hotel_name as hotels,h.hotel_id as id"],
       ["Best hotels in Tokyo with free wifi and swimming pool", "(h:hotel) WHERE h.city='Tokyo' AND h.amenities CONTAINS 'Free Wi-Fi' AND h.amenities CONTAINS 'Pool' RETURN h.hotel_name as hotels,h.hotel_id as id"],
       ["Hotels in tokyo with a single room with a balcony", "(h:hotel) WHERE h.city='Tokyo' AND h.accomodation CONTAINS 'Single Room' AND h.special_features CONTAINS 'Balcony/Patio' RETURN h.hotel_name as hotels,h.hotel_id as id"]
        
        Note: Do not include any explanations or apologies in your responses. Return just the cypher query without any quotation marks. Return hotel_name and hotel_id.
        """

    def query_database(self, neo4j_query, params={}):

        '''
        executes query in database
    
        Parameters
        --------------
        neo4j_query: query string containing cypher statements
        params: parameters to be passed along with query 
            
        Returns: result of the query execution
        '''

        with self.driver.session() as session:
            result = session.run(neo4j_query, params)
            output = [r.values() for r in result]
            output.insert(0, result.keys())
            return output

    def construct_cypher(self, question, history=None):

        '''
        convert the natural language string into cypher statements with the help of openai
        model gpt
    
        Parameters
        --------------
        question: natural language string
        history: if the function is called in case of previous incorrect(syntax error) cypher
        generation, we can pass previous incorrect cypher and instructions associated with it
        
        Returns: cypher statements string 

        '''

        messages = [
            {"role": "system", "content": self.get_system_message()},
            {"role": "user", "content": question},
        ]
        # Used for Cypher healing flows
        if history:
            messages.extend(history)
        client = OpenAI(api_key=self.api_key)
        completions = client.chat.completions.create(
            model="gpt-4",
            temperature=0.0,
            max_tokens=1000,
            messages=messages
        )
        return completions.choices[0].message.content

    def run(self, question, history=None, retry=False):

        '''
        main function responsible for constructing cypher and controls the self healing flows
        of incorrect cypher statements
    
        Parameters
        --------------
        question: natural language string
        history: containing instructions and previously generated incorrect cypher to be corrected in this attempt  
            
        Returns: 
        calls query_database function in case of succesful cypher generation
        '''

        # Construct Cypher statement
        cypher = self.construct_cypher(question, history)
        print(cypher)
        try:
            return self.query_database(cypher)
        # Self-healing flow
        except CypherSyntaxError as e:
            # If out of retries
            if not retry:
              return "Invalid Cypher syntax"
        # Self-healing Cypher flow by
        # providing specific error to GPT-4
            print("Retrying")
            return self.run(
                question,
                [
                    {"role": "assistant", "content": cypher},
                    {
                        "role": "user",
                        "content": f"""This query returns an error: {str(e)} 
                        query should contain only cypher statements. remove explanations or apologies""",
                    },
                ],
                retry=False
            )

    def __del__(self):
        #closes the connection with the database
        if self.driver:
            self.driver.close()


In [7]:
gds_db = Neo4jGPTQuery(
    url=url_neo4j,
    user=user_neo4j,
    password=pass_neo4j,
    openai_api_key=openaikey,
)

In [8]:
result =gds_db.run("Child friendly budget motels in Madrid")
print(result)

MATCH (h:hotel) WHERE h.city='Madrid' AND h.property_type='Motel' AND (h.cost_range='0-1500' OR h.cost_range='1500-3000' OR h.cost_range='3000-6000') AND h.family_friendly_amenities IS NOT NULL RETURN h.hotel_name as hotels,h.hotel_id as id
[['hotels', 'id'], ['Soothing Mountain Island', 461], ['Whispering Wonderland', 932], ['Vista Sanctum', 1593], ['Springs Majestic Chateau', 1708], ['Picturesque Island', 6129], ['Mellow Stellar Bungalow', 6161], ['Wind Velvety Territory', 11730], ['Alluring Hostel', 15704], ['Ruby Vista Loft', 18261], ['Mountain Idyllic Voyage', 36455]]
